# Binh and Korn function multi-objective optimization (min).

Minimize the equations given by:

\begin{cases}
      f_{1}\left(x,y\right) = 4x^{2} + 4y^{2}, \\
      f_{2}\left(x,y\right) = \left(x - 5\right)^{2} + \left(y - 5\right)^{2} \\
\end{cases}

subject to:

\begin{cases}
      g_{1}\left(x,y\right) = \left(x - 5\right)^{2} + y^{2} \leq 25, \\
      g_{2}\left(x,y\right) = \left(x - 8\right)^{2} + \left(y + 3\right)^{2} \geq 7.7 \\
\end{cases}

where:

\begin{cases}
      0\le x \le 5, \\
      0\le y \le 3. \\
\end{cases}

The Pareto-optimal solutions are constituted by solutions:
    $ x=y ∈ [0.0, 3.0] \text{  and  } x ∈ [3.0, 5.0], y=3.0.$

### First we import python libraries and set up the directory of our code.

In [1]:
import os, sys
import numpy as np
from math import fsum
import matplotlib.pyplot as plt

PROJECT_DIR = os.path.abspath('../code/')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [2]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

### Define the sphere function, which plays also the role of the 'fitness' function.

In addition, we define the '_func' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [3]:
# Multi-objective function.
def fun_Binh_Korn(individual: Chromosome, f_min: bool = True):
    
    # Set the penalty coefficient.
    p_c = 0.5
    
    # Extract gene values as 'x' and 'y', for parsimony.
    x, y = individual[0].datum, individual[1].datum
    
    # Compute each objective function.
    f1 = 4.0 * (x**2 + y**2)
    f2 = (x - 5.0)**2 + (y - 5.0)**2
    
    # Compute the constraints.
    g1 = max(0.0, (x - 5.0)**2 + y**2 - 25.0)**2
    g2 = min(0.0, (x - 8.0)**2 + (y + 2.0)**2 - 7.7)**2
    
    # Compute the final value.
    f_val = np.mean([f1 + p_c*g1,
                     f2 + p_c*g2])
    
    # Return the negative (to account for minimization).
    return -f_val if f_min else f_val
# _end_def_

# Random function: ~Ux(0.0, 5.0).
_func_x = lambda: np.random.uniform(0.0, 5.001)

# Random function: ~Uy(0.0, 3.0).
_func_y = lambda: np.random.uniform(0.0, 3.001)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by '_func'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [4]:
# Define the number of chromosomes.
N = 100

# Initial population.
population = [Chromosome([Gene(np.random.uniform(0.0, 5.001), _func_x),
                          Gene(np.random.uniform(0.0, 3.001), _func_y)], np.nan, True)
              for i in range(N)]

# Create the StandardGA object that will carry on the optimization.
# Note: in this example we have accepted, silently, the default probabilities in the
# genetic operators. These can be altered here before passed to the StandardGA constructor.
toy_GA = StandardGA(initial_pop=population, fit_func=fun_Binh_Korn, select_op=LinearRankSelector(),
                    mutate_op=RandomMutator(), cross_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [5]:
toy_GA.run(epochs=5000, elitism=True, f_tol=1.0e-6, verbose=False)

Initial Avg. Fitness = -33.7757
Final   Avg. Fitness = -20.5468
Elapsed time: 73.720 seconds.


In [6]:
# Extract the optimal values.
x, y = toy_GA.best_chromosome()._genome

# Compute the final objective functions.
f1 = 4.0 * (x.datum**2 + y.datum**2)
f2 = (x.datum - 5.0)**2 + (y.datum - 5.0)**2

# Print the resutls.
print(f"x={x.datum:.5f}, y={y.datum:.5f}", end='\n\n')
print(f"f1(x,y) = {f1:.5f}")
print(f"f2(x,y) = {f2:.5f}")

x=1.00020, y=1.00000

f1(x,y) = 8.00162
f2(x,y) = 31.99838


**Note that the above solution is indeed in the Pareto-optimal solutions!**

### End of file